In [85]:
import time
import statistics
import serial
import serial.tools.list_ports
import ipywidgets as widgets


In [87]:
com_ports=serial.tools.list_ports.comports()
ports=[]
for p in com_ports:
    ports.append(p.device)

com_port=widgets.Dropdown(
    options=ports,
    description='Port:',
    disabled=False,
)
display(com_port)

Dropdown(description='Port:', options=('/dev/ttyS31', '/dev/ttyS30', '/dev/ttyS29', '/dev/ttyS28', '/dev/ttyS2…

In [84]:
arduino_connect_button = widgets.Button(
    description='Connetti Arduino',
    disabled=False,
    button_style='', 
    tooltip='Click me',
    icon='plug' 
)
output = widgets.Output(layout={'border': '2px solid black'})

arduino=None
def connect_arduino(b):
    global arduino
    arduino=serial.Serial(com_port.value, baudrate=115200, timeout=1)
    test=2
    with output:
        print("Arduino connected")

arduino_connect_button.on_click(connect_arduino)
display(arduino_connect_button,output)


Button(description='Connetti Arduino', icon='plug', style=ButtonStyle(), tooltip='Click me')

Output(layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right='2px solid b…

In [53]:
handshake_button = widgets.Button(
    description='Handshake Arduino',
    disabled=False,
    button_style='', 
    tooltip='Click me',
    icon='handshake'
)
handshake_output = widgets.Output(layout={'border': '2px solid black'})
handshake_output.clear_output()

def handshake_arduino(sleep_time=1):

    timeout = arduino.timeout
    arduino.timeout = 2
    arduino.write('h'.encode())
    
    while (arduino.in_waiting < 0):
        pass
    
    response= arduino.read_until()
    arduino.timeout = timeout
    with handshake_output:
        
        print(response.decode().strip())
        
handshake_button.on_click(handshake_arduino)
display(handshake_button,handshake_output)


Button(description='Handshake Arduino', icon='handshake', style=ButtonStyle(), tooltip='Click me')

Output(layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right='2px solid b…

In [68]:
def parse_raw_line(raw_line):
    raw_line = raw_line.decode()
    if raw_line[-1] != "\n":
        raise ValueError(
            "Il messaggio inviato da Arduino deve terminare con il carattere nuova linea."
        )
    
    broadband,ir= raw_line.rstrip().split("|")

    return int(broadband),int(ir)

In [77]:
samples=widgets.Dropdown(
    options=range(1,16),
    description='Samples:',
    disabled=False,
)

integration_time=widgets.Dropdown(
                    options=[13,102,402],
                    value=402,
                    description='Samples:',
                    disabled=False,
)
measure_output = widgets.Output(layout={'border': '2px solid black'})

TSL2561_CLIPPING_402MS:int=65000
TSL2561_LUX_CHSCALE:int=10
TSL2561_LUX_RATIOSCALE:int =9

def calculateLux(broadband:int,ir:int):
    clipThreshold = TSL2561_CLIPPING_402MS
    chScale = (1 << TSL2561_LUX_CHSCALE)
    channel0:int = (broadband * chScale) >> TSL2561_LUX_CHSCALE
    channel1:int = (ir * chScale) >> TSL2561_LUX_CHSCALE
    ratio:float=0
    lux:float=0
    if (channel0 != 0):
        ratio =channel1/channel0
        
    if ratio>0 and ratio <=0.5:
        lux=0.0304*channel0-0.062*channel0*pow(ratio,1.4)
    elif ratio>0.5 and ratio<=0.61:
        lux=0.0224*channel0-0.031*channel1
    elif ratio>0.61 and ratio <=0.80:
        lux=0.0128*channel0-0.0153*channel1
    elif ratio>0.80 and ratio <=1.30:
        lux=0.00146*channel0-0.00112*channel1
    elif ratio>1.30 and ratio <=3.00:
        lux=0.0
    
    return round(lux,1)


@measure_output.capture()
def get_light_reading(b):
    #n_samples: average over n_samples
    #sensor integration time, defaults to 402ms
    broadband_readings=[]
    ir_readings=[]
    n_samples=samples.value
    int_time=integration_time.value
    
    for reading in range(n_samples):
            # invio il comando per la lettura del sensore TSL2561
            arduino.write('r'.encode())
            while arduino.in_waiting < 0:
                pass
            # Leggo i dati inviati tramite seriale a seguito della richiesta di lettura
            raw = arduino.read_until()
    
            broadband,ir=parse_raw_line(raw)
            broadband_readings.append(broadband)
            ir_readings.append(ir)
            time.sleep(1.5*int_time/1000)
        
    lux_data=calculateLux(int(statistics.mean(broadband_readings)),int(statistics.mean(ir_readings)))
    print(lux_data)
    return lux_data

measure_button = widgets.Button(
    description='Measure light',
    disabled=False,
    button_style='', 
    tooltip='Click me',
    icon='ruler'
)

measure_button.on_click(get_light_reading)


display(samples,integration_time,measure_button,measure_output)
    

Dropdown(description='Samples:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15), value=1)

Dropdown(description='Samples:', index=2, options=(13, 102, 402), value=402)

Button(description='Measure light', icon='ruler', style=ButtonStyle(), tooltip='Click me')

Output(layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right='2px solid b…